In [1]:
import pandas as pd

# ---------- Commuting-flow file ----------
flows = pd.read_csv(
    "ACS_Commuting_Flows.csv",
    dtype=str                     # keep codes zero-padded
)
flows = flows.dropna(subset=['ResidenceState', 'ResidenceCounty', 'WorkplaceState', 'WorkplaceCounty'])
# make 5-digit county FIPS for both ends of the trip
flows["res_fips"] = flows["ResidenceState"].str.zfill(2).str[-2:] + flows["ResidenceCounty"].str.zfill(3)
flows["wrk_fips"] = flows["WorkplaceState"].str.zfill(2).str[-2:] + flows["WorkplaceCounty"].str.zfill(3)

# convert counts to numeric
flows["flow"] = flows["Workers in Commuting Flow"].astype(int)
flows = flows[flows["flow"] > 0]                          # drop empty flows
flows

,ResidenceState,ResidenceCounty,WorkplaceState,WorkplaceCounty,Workers in Commuting Flow,Margin of Error,res_fips,wrk_fips,flow
0,01,001,001,001,8671,681,01001,01001,8671
1,01,001,001,003,8,12,01001,01003,8
2,01,001,001,007,11,19,01001,01007,11
3,01,001,001,013,9,17,01001,01013,9
4,01,001,001,021,508,195,01001,01021,508
...,...,...,...,...,...,...,...,...,...
122330,72,153,072,123,38,37,72153,72123,38
122331,72,153,072,125,86,62,72153,72125,86
122332,72,153,072,127,141,109,72153,72127,141
122333,72,153,072,149,8,13,72153,72149,8


In [2]:
xwalk = (
    pd.read_csv("/net/dali/home/mscbio/rul98/TrendDetection/geocorr2014.csv", dtype=str)
      .loc[:, ["county", "hrr", "afact"]]
      .assign(afact=lambda d: pd.to_numeric(d["afact"], errors="coerce"))
      .query("afact > 0")             # exclude zero-share fragments
)
xwalk = xwalk.dropna()
xwalk["afact"] = xwalk.groupby("county")["afact"].transform(lambda s: s / s.sum())
xwalk

,county,hrr,afact
1,01001,001,0.053
2,01001,007,0.947
3,01003,006,0.960
4,01003,134,0.040
5,01005,002,1.000
...,...,...,...
4666,56039,274,0.004
4667,56039,423,0.996
4668,56041,423,1.000
4669,56043,274,1.000


In [3]:
flows = flows.merge(
    xwalk.rename(columns={"hrr": "res_hrr", "afact": "res_afact"}),
    left_on="res_fips", right_on="county",
    how="left",
    suffixes=("", "_res")
)
flows = flows.dropna()
flows = flows.merge(
    xwalk.rename(columns={"hrr": "wrk_hrr", "afact": "wrk_afact"}),
    left_on="wrk_fips", right_on="county",
    how="left",
    suffixes=("", "_wrk")
)
flows = flows.dropna()

In [4]:
flows.drop_duplicates()

,ResidenceState,ResidenceCounty,WorkplaceState,WorkplaceCounty,Workers in Commuting Flow,Margin of Error,res_fips,wrk_fips,flow,county,res_hrr,res_afact,county_wrk,wrk_hrr,wrk_afact
0,01,001,001,001,8671,681,01001,01001,8671,01001,001,0.053,01001,001,0.053
1,01,001,001,001,8671,681,01001,01001,8671,01001,001,0.053,01001,007,0.947
2,01,001,001,001,8671,681,01001,01001,8671,01001,007,0.947,01001,001,0.053
3,01,001,001,001,8671,681,01001,01001,8671,01001,007,0.947,01001,007,0.947
4,01,001,001,003,8,12,01001,01003,8,01001,001,0.053,01003,006,0.960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287606,56,045,056,005,595,163,56045,56005,595,56045,457,1.000,56005,274,1.000
287607,56,045,056,011,74,46,56045,56011,74,56045,457,1.000,56011,274,0.013
287608,56,045,056,011,74,46,56045,56011,74,56045,457,1.000,56011,370,0.229
287609,56,045,056,011,74,46,56045,56011,74,56045,457,1.000,56011,457,0.758


In [5]:
flows = flows.drop_duplicates()
flows["flow_hrr2hrr"] = (
    flows["flow"]           # original ACS worker count
  * flows["res_afact"]         # share of origin-county population in res_hrr
  * flows["wrk_afact"]         # share of dest-county population in wrk_hrr
)
hrr2hrr = (
    flows.groupby(["res_hrr", "wrk_hrr"], as_index=False)
         .agg(flow = ("flow_hrr2hrr", "sum"))
)

In [6]:
hrr2hrr

,res_hrr,wrk_hrr,flow
0,001,001,845774.387084
1,001,002,290.628880
2,001,005,44974.506751
3,001,006,2526.221014
4,001,007,15875.268989
...,...,...,...
41494,457,423,832.084136
41495,457,432,9.000000
41496,457,442,35.188000
41497,457,445,34.036000


In [7]:
hrr_matrix = (
    hrr2hrr.pivot(index="res_hrr", columns="wrk_hrr", values="flow")
           .fillna(0)
)
hrr_matrix

wrk_hrr,001,002,005,006,007,009,010,011,012,014,...,445,446,447,448,449,450,451,452,456,457
res_hrr,,,,,,,,,,,,,,,,,,,,,
001,845774.387084,290.628880,44974.506751,2526.221014,15875.268989,15952.642290,17.359000,16.880640,34.745726,4.139520,...,11.972,0.534768,13.115232,0.000000,0.073440,0.000000,4.974092e+01,0.000000,0.608000,0.000000
002,522.448578,126669.049535,28.085000,113.326182,3508.645874,70.115116,0.000000,4.710000,9.295000,1.155000,...,0.352,0.024000,5.976000,0.000000,0.000000,0.000000,1.000000e+01,0.000000,0.000000,0.000000
005,26211.834435,45.442000,268957.421670,50.709770,163.387115,62.129778,0.011988,9.522050,19.085380,2.335025,...,0.000,0.459600,0.482400,0.000000,0.000000,0.000000,1.410923e+01,0.000000,0.000000,0.000000
006,2436.169391,92.791136,72.311688,313272.161870,881.489096,423.809619,26.482000,0.000000,0.180000,0.000000,...,0.000,0.130760,32.559240,12.104640,0.000000,0.000000,1.076044e+01,0.000000,0.000000,0.000000
007,12649.968587,1295.563634,29.297426,916.745450,180483.747295,127.527266,82.389000,24.247000,33.766944,2.695000,...,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,3.988000e+00,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,1.815660,0.000000,0.000000,0.401799,0.000000,0.000000,16.248438,0.599112,1.183857,0.146916,...,0.000,2419.102237,347.387667,2650.724751,2320.663913,126892.477565,6.267057e+02,1994.971270,20881.244510,0.000000
451,0.000000,0.000000,0.000000,7.252747,9.000000,0.000000,1.970000,39.386353,76.541713,9.658437,...,0.000,1932.089267,5962.289431,242.021358,28565.491526,473.420369,1.172862e+06,11460.722051,136.606773,44.000000
452,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.964000,0.824716,1.599529,0.202239,...,0.000,21272.051973,2859.589107,107.627977,7446.585379,1836.079956,1.297837e+04,74342.535254,33.333854,0.000000


In [8]:
hrr_matrix.to_csv('HHS_Flows.csv')